In [57]:
# %load_ext autorelaod
# %autoreload 2

import sys
sys.path.append('./../RWKV-v4neo/src')
import torch 
from torch.utils import data
from enwik8_dataset import EnWik8
import json
import argparse
import os
from torch import nn

In [25]:
def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)
def set_environ(env_vars: dict):
    for k, v in env_vars.items():
        os.environ[k] = v

In [26]:
data_path = '/system/user/beck/pwbeck/projects/rwkv/RWKV-LM/data/enwik8'
args = load_json('./debug_args.json')
args = argparse.Namespace(**args)
env_vars = load_json('./debug_environ.json')
set_environ(env_vars)

In [34]:
batch_size = 12

In [49]:
dataset = EnWik8(datafile=data_path)
dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True, drop_last=True)
args.vocab_size = dataset.vocab_size

Data has 99621832 tokens, 6064 vocab size.


In [39]:
batch = next(iter(dataloader))
x, y = batch
x.shape, y.shape

(torch.Size([12, 512]), torch.Size([12, 512]))

In [56]:
from model import RWKV, Block, RWKV_ChannelMix, RWKV_TimeMix
model = RWKV(args)

In [53]:
output = model(x)

In [54]:
output.shape

torch.Size([12, 512, 6064])

In [55]:
args.head_qk

0

In [68]:
zp = nn.ZeroPad2d((0, 0, 1, -1))

In [69]:
t = torch.randn(2, 10, 5)
t

tensor([[[ 2.2994,  0.6164, -0.2690,  0.4272,  0.0744],
         [-0.9279, -1.5175,  1.2302,  1.2907, -1.4773],
         [ 0.1119,  2.7972,  0.2340, -0.5418,  0.3708],
         [ 0.8383, -0.6947,  0.1652, -1.0719, -1.8091],
         [ 0.1493, -0.9867,  0.2290,  1.0927, -0.1701],
         [ 0.7025, -0.6657, -1.0994, -0.7444,  0.5355],
         [-1.8331, -0.4958,  0.3811, -0.1416,  0.0564],
         [-1.6845,  1.4969,  2.0742, -0.6748,  2.6469],
         [ 1.3789,  0.0776,  1.1843,  0.2751,  1.3776],
         [-1.4278, -0.5132, -0.4568, -0.7256, -0.4097]],

        [[ 0.6982, -0.5857, -0.3218, -2.6634,  0.6578],
         [ 0.5408, -0.4575,  0.2834,  1.1029,  0.1854],
         [-0.6689,  1.0695, -2.1253, -0.5972,  0.2518],
         [-1.8999, -0.2150, -1.3322,  1.5590, -0.9475],
         [ 0.1050, -0.3192,  0.0122, -0.4665, -0.7289],
         [ 1.3017, -1.0393, -1.3678, -0.2464, -0.7555],
         [-1.3445,  1.5817, -0.5817,  0.5974, -0.0958],
         [ 0.0687,  0.1729, -0.6784,  0.4652, 

In [70]:
zp(t).shape

torch.Size([2, 10, 5])

In [71]:
zp(t)

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 2.2994,  0.6164, -0.2690,  0.4272,  0.0744],
         [-0.9279, -1.5175,  1.2302,  1.2907, -1.4773],
         [ 0.1119,  2.7972,  0.2340, -0.5418,  0.3708],
         [ 0.8383, -0.6947,  0.1652, -1.0719, -1.8091],
         [ 0.1493, -0.9867,  0.2290,  1.0927, -0.1701],
         [ 0.7025, -0.6657, -1.0994, -0.7444,  0.5355],
         [-1.8331, -0.4958,  0.3811, -0.1416,  0.0564],
         [-1.6845,  1.4969,  2.0742, -0.6748,  2.6469],
         [ 1.3789,  0.0776,  1.1843,  0.2751,  1.3776]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.6982, -0.5857, -0.3218, -2.6634,  0.6578],
         [ 0.5408, -0.4575,  0.2834,  1.1029,  0.1854],
         [-0.6689,  1.0695, -2.1253, -0.5972,  0.2518],
         [-1.8999, -0.2150, -1.3322,  1.5590, -0.9475],
         [ 0.1050, -0.3192,  0.0122, -0.4665, -0.7289],
         [ 1.3017, -1.0393, -1.3678, -0.2464, -0.7555],
         [-1.3445,  1.5817, -0.5817,  0.5974, 

In [74]:
zp(t)[0, 1, :]

tensor([ 2.2994,  0.6164, -0.2690,  0.4272,  0.0744])

In [73]:
t[0, 0, :]

tensor([ 2.2994,  0.6164, -0.2690,  0.4272,  0.0744])